In [ ]:
!pip3 install statsmodels

In [625]:
import numpy as np
import pandas as pd

from datetime import date, timedelta
import datetime

import matplotlib.dates as mdates
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
from matplotlib.lines import Line2D

from sklearn.preprocessing import LabelEncoder

import statsmodels.api as sm

In [626]:
snapshot_ids = []

def convertToTime(row, columnName):
    return datetime.datetime.strptime(row[columnName], "%Y-%m-%d").date()

def convertToDate(row, columnName):
    return datetime.datetime.strptime(row[columnName], "%Y-%m-%d %H:%M:%S").date()

def getDays(row, beginColumnName, endColumnName):
    v = datetime.datetime.strptime(row[endColumnName], "%Y-%m-%d").date() - datetime.datetime.strptime(row[beginColumnName], "%Y-%m-%d").date()
    return v.days

def lookup_index(row, columnName, array):
    if(row[columnName] not in array):
        return -1
    return array.index(row[columnName]) + 1

def colour_life_events(row):
    colours = {'personal':'lightcoral', 'health':'orange', 'work':'lightgreen', 'financial':'teal', 'weather':'blueviolet', 'societal':'navy','other':'skyblue'}
    return colours[row['life_event_type']]

def remove_rows(base_df, other_df):
    modified_df = other_df.drop(other_df[other_df['snapshot_id'] not in base_df['snapshot_id'].values].index)
    return modified_df

In [627]:
demographics_data = pd.read_csv('data/igtbs_demographics_complete.csv', parse_dates=True)
snapshot_ids = list(demographics_data['snapshot_id'].values)
demographics_data = demographics_data[['age','gender','snapshot_id', 'shipley.vocab', 'shipley.abs', 'openness', 'conscientiousness', 'extraversion', 'agreeableness', 'neuroticism','pos.affect','neg.affect','stai.trait']]
demographics_data['y_axis'] = demographics_data.apply(lookup_index, columnName='snapshot_id', array=snapshot_ids, axis=1)

demographics_data

,age,gender,snapshot_id,shipley.vocab,shipley.abs,openness,conscientiousness,extraversion,agreeableness,neuroticism,pos.affect,neg.affect,stai.trait,y_axis
0,39.0,Female,11156103277197680506,32.0,15.0,4.750000,4.666667,3.666667,4.416667,2.333333,38.0,15.0,39.0,1
1,39.0,Female,13346780717560972831,38.0,19.0,3.500000,4.833333,3.000000,4.916667,1.833333,41.0,16.0,28.0,2
2,25.0,Female,12932409338132389043,32.0,19.0,4.916667,4.083333,3.750000,4.000000,4.083333,20.0,24.0,55.0,3
3,33.0,Male,10385244076367909694,34.0,17.0,3.416667,2.166667,2.916667,3.000000,2.000000,25.0,24.0,51.0,4
4,37.0,Female,10671464268435700062,39.0,15.0,4.500000,5.000000,3.916667,3.750000,3.000000,36.0,16.0,35.0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
749,44.0,Male,10033784408945437200,34.0,17.0,3.500000,4.250000,2.750000,4.500000,2.166667,30.0,13.0,35.0,750
750,51.0,Male,13257691313989387573,34.0,20.0,4.500000,2.666667,2.750000,4.250000,2.833333,35.0,17.0,41.0,751
751,44.0,Female,13370808519384713343,35.0,18.0,3.500000,4.250000,4.333333,3.250000,1.083333,39.0,12.0,25.0,752
752,31.0,Female,12173290729829297463,32.0,17.0,3.583333,4.666667,2.916667,3.416667,3.166667,38.0,18.0,31.0,753


In [628]:
df_life_events = pd.read_csv('data/Superimposed/LifeEvents_Curated_non_blinded.csv', parse_dates=True)
df_life_events = df_life_events[['snapshot_id', 'description','UpdatedBeginDate', 'UpdatedEndDate', 'date_confidence','life_event_type', 'work_perf_impact', 'significance','valence', 'ended_or_ongoing']]
df_life_events = df_life_events.drop(df_life_events[((df_life_events['UpdatedBeginDate'].isnull() == True) | (df_life_events['UpdatedEndDate'].isnull() == True))].index)
df_life_events.replace({'valence': {np.nan: 'Neither Positive or Negative'}, 'significance': {np.nan: 'Neither Positive or Negative'}, 'date_confidence': {np.nan: 'Moderate confidence'}}, inplace=True)

df_life_events['num_of_days'] = df_life_events.apply(getDays, endColumnName='UpdatedEndDate', beginColumnName='UpdatedBeginDate', axis=1)
df_life_events['UpdatedBeginDate_time'] = df_life_events.apply(convertToTime, columnName='UpdatedBeginDate', axis=1)
df_life_events['y_axis'] = df_life_events.apply(lookup_index, columnName='snapshot_id', array=snapshot_ids, axis=1)
df_life_events['colour'] = df_life_events.apply(colour_life_events, axis=1)

df_life_events

,snapshot_id,description,UpdatedBeginDate,UpdatedEndDate,date_confidence,life_event_type,work_perf_impact,significance,valence,ended_or_ongoing,num_of_days,UpdatedBeginDate_time,y_axis,colour
0,12516215411311181588,Took a vacation,2018-07-07,2018-07-28,Highest confidence,personal,Small positive effect,Moderate significance,Very Positive,Ended,21,2018-07-07,364,lightcoral
1,12131199544704830706,Vacation,2018-05-12,2018-05-28,Highest confidence,personal,Small positive effect,Moderate significance,Extremely Positive,Ended,16,2018-05-12,370,lightcoral
2,10218374518126117174,car crash,2018-11-15,2018-12-15,High confidence,personal,No effect,Moderately low significance,Somewhat Negative,Ended,30,2018-11-15,366,lightcoral
3,12680617526080257161,I moved to tampa florida in APril 2019,2018-10-18,2019-04-10,Highest confidence,personal,Large positive effect,High significance,Extremely Positive,Ended,174,2018-10-18,373,lightcoral
4,13467047711801587306,Vacation,2018-11-01,2018-12-01,Highest confidence,personal,Medium positive effect,Moderately low significance,Somewhat Positive,Ended,30,2018-11-01,413,lightcoral
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1190,12718614204893805965,bitter cold - pipes in house froze,2019-02-01,2019-03-15,Moderate confidence,weather,NaN,Very Negative,Moderately high significance,Ended,42,2019-02-01,592,blueviolet
1191,9883624450358368517,Below 0 temps-polar vortex,2019-01-29,2019-02-03,Moderately low confidence,weather,NaN,Very Negative,Moderately high significance,Ended,5,2019-01-29,486,blueviolet
1192,12994028865111584912,Cold weather,2019-01-05,2019-01-05,Moderate confidence,weather,NaN,Somewhat Positive,High significance,Ended,0,2019-01-05,476,blueviolet
1193,13146988918954476898,Blizzard,2019-01-27,2019-01-29,High confidence,weather,NaN,Somewhat Negative,Moderate significance,Ended,2,2019-01-27,602,blueviolet


In [629]:
def fix_signficance(row):
    if('significance' in row['valence']):
        return row['valence']
    else:
        return row['significance']

def fix_valence(row):
    if('significance' in row['significance']):
        return row['valence']
    else:
        return row['significance']    

df_life_events['fixed_signficance'] = df_life_events.apply(fix_signficance, axis = 1)
df_life_events['fixed_valence'] = df_life_events.apply(fix_valence, axis = 1)
df_life_events = df_life_events.drop(columns = ['valence', 'significance'])
df_life_events = df_life_events.rename(columns={"fixed_signficance": "significance", "fixed_valence": "valence"})
df_life_events

,snapshot_id,description,UpdatedBeginDate,UpdatedEndDate,date_confidence,life_event_type,work_perf_impact,ended_or_ongoing,num_of_days,UpdatedBeginDate_time,y_axis,colour,significance,valence
0,12516215411311181588,Took a vacation,2018-07-07,2018-07-28,Highest confidence,personal,Small positive effect,Ended,21,2018-07-07,364,lightcoral,Moderate significance,Very Positive
1,12131199544704830706,Vacation,2018-05-12,2018-05-28,Highest confidence,personal,Small positive effect,Ended,16,2018-05-12,370,lightcoral,Moderate significance,Extremely Positive
2,10218374518126117174,car crash,2018-11-15,2018-12-15,High confidence,personal,No effect,Ended,30,2018-11-15,366,lightcoral,Moderately low significance,Somewhat Negative
3,12680617526080257161,I moved to tampa florida in APril 2019,2018-10-18,2019-04-10,Highest confidence,personal,Large positive effect,Ended,174,2018-10-18,373,lightcoral,High significance,Extremely Positive
4,13467047711801587306,Vacation,2018-11-01,2018-12-01,Highest confidence,personal,Medium positive effect,Ended,30,2018-11-01,413,lightcoral,Moderately low significance,Somewhat Positive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1190,12718614204893805965,bitter cold - pipes in house froze,2019-02-01,2019-03-15,Moderate confidence,weather,NaN,Ended,42,2019-02-01,592,blueviolet,Moderately high significance,Very Negative
1191,9883624450358368517,Below 0 temps-polar vortex,2019-01-29,2019-02-03,Moderately low confidence,weather,NaN,Ended,5,2019-01-29,486,blueviolet,Moderately high significance,Very Negative
1192,12994028865111584912,Cold weather,2019-01-05,2019-01-05,Moderate confidence,weather,NaN,Ended,0,2019-01-05,476,blueviolet,High significance,Somewhat Positive
1193,13146988918954476898,Blizzard,2019-01-27,2019-01-29,High confidence,weather,NaN,Ended,2,2019-01-27,602,blueviolet,Moderate significance,Somewhat Negative


In [630]:
df_facebook = pd.read_csv('data/Superimposed/Facebook Data For Life Events-Combined - FB Data.csv')
df_facebook = df_facebook[['snapshot_id', 'date_enrolled', 'created_time', 'Text', 'final_life_event_category', 'final_life_event_category_2']]
df_facebook = df_facebook.replace({'PostiveMove':'PositiveMove', 'Negative Move':'NegativeMove'})

df_facebook = df_facebook.drop(df_facebook[((df_facebook['final_life_event_category'].isnull() == True) & (df_facebook['final_life_event_category_2'].isnull() == True))].index)
df_facebook['y_axis'] = df_facebook.apply(lookup_index, columnName='snapshot_id', array=snapshot_ids, axis=1)
df_facebook['created_date'] = df_facebook.apply(convertToDate, columnName='created_time', axis=1)
df_facebook

,snapshot_id,date_enrolled,created_time,Text,final_life_event_category,final_life_event_category_2,y_axis,created_date
0,12840161237957324034,2018-03-02 0:00:00,2018-03-16 17:30:07,"Next stop....Seoul, South Korea!",Vacation,Vacation,302,2018-03-16
1,11156103277197680506,2018-06-12 0:00:00,2018-07-31 13:10:19,¡Hola Barcelona! We are in you 😂,Vacation,Vacation,1,2018-07-31
2,11329586216091316598,2018-06-12 0:00:00,2018-09-08 2:11:19,Family beach vacation has begun!! Day one was...,Vacation,Vacation,478,2018-09-08
3,13543308681200334026,2018-06-26 0:00:00,2018-07-22 16:05:38,"🛫 to sea, sand and scuba",Vacation,Vacation,736,2018-07-22
4,12676581714475000627,2018-03-27 0:00:00,2018-04-25 23:55:28,Off to Dublin!! #fernweh #wanderlust #selfdisc...,Vacation,Vacation,474,2018-04-25
...,...,...,...,...,...,...,...,...
14181,12971652107468080963,2018-04-24 0:00:00,2018-08-26 18:56:39,Anthony is officially a middle schooler! His ...,ChangedSchool,ChangedSchool,577,2018-08-26
14182,12971652107468080963,2018-04-24 0:00:00,2018-08-26 22:25:53,On Friday we closed on our home and it brings ...,NeutralMove,NeutralMove,577,2018-08-26
14184,12971652107468080963,2018-04-24 0:00:00,2018-10-03 0:45:06,These handsome fellas!!! Adam: Boulder High S...,ChangedSchool,ChangedSchool,577,2018-10-03
14188,12201010563123762054,2018-02-26 0:00:00,2018-03-02 14:27:59,Our house is on the market! If you know anyone...,NeutralMove,NeutralMove,246,2018-03-02


In [631]:
df_stress = pd.read_csv('data/Superimposed/dailies_scores.csv', low_memory=False)
df_stress = df_stress[['snapshot_id','day', 'alc_status', 'alc.quantity.d', 'anxiety.d', 'pos.affect.d', 'neg.affect.d','sleep.d', 'stress.d']]
print(df_stress['stress.d'].value_counts())

df_stress['y_axis'] = df_stress.apply(lookup_index, columnName='snapshot_id', array=snapshot_ids, axis=1)
df_stress['day_time'] = df_stress.apply(convertToTime, columnName='day', axis=1)
df_stress

1.0    14043
2.0    13233
3.0     9441
4.0     1493
5.0      256
Name: stress.d, dtype: int64


,snapshot_id,day,alc_status,alc.quantity.d,anxiety.d,pos.affect.d,neg.affect.d,sleep.d,stress.d,y_axis,day_time
0,10003528337325499062,2018-03-01,yes,3.0,2.0,11.0,6.0,6.0,2.0,265,2018-03-01
1,10003528337325499062,2018-03-02,no,0.0,2.0,13.0,5.0,8.0,2.0,265,2018-03-02
2,10003528337325499062,2018-03-04,no,0.0,1.0,5.0,5.0,9.0,1.0,265,2018-03-04
3,10003528337325499062,2018-03-06,no,0.0,1.0,15.0,5.0,8.0,1.0,265,2018-03-06
4,10003528337325499062,2018-03-09,no,0.0,2.0,11.0,5.0,8.0,1.0,265,2018-03-09
...,...,...,...,...,...,...,...,...,...,...,...
38592,9999689421615292586,2018-06-23,NaN,NaN,1.0,8.0,5.0,NaN,1.0,606,2018-06-23
38593,9999689421615292586,2018-06-25,NaN,NaN,1.0,5.0,5.0,NaN,1.0,606,2018-06-25
38594,9999689421615292586,2018-06-28,NaN,NaN,2.0,12.0,5.0,NaN,2.0,606,2018-06-28
38595,9999689421615292586,2018-06-30,NaN,NaN,1.0,8.0,5.0,NaN,1.0,606,2018-06-30


In [632]:
# fig, ax = plt.subplots(figsize=(40,40))

# ymin = min(df_life_events['y_axis'].values)
# ymax = min(max(df_facebook['y_axis'].values), max(df_life_events['y_axis'].values))

# start = [mdates.date2num(v) for v in df_life_events['UpdatedBeginDate_time'].values]

# xmin = max(min(df_facebook['created_date'].values), min(df_life_events['UpdatedBeginDate_time'].values))
# xmax = min(max(df_facebook['created_date'].values), max(df_life_events['UpdatedBeginDate_time'].values))
        
# plt.plot(df_facebook['created_date'].values, df_facebook['y_axis'].values, linestyle = 'None', marker='*', color='black', label='line with marker')
# plt.barh(y=df_life_events['y_axis'].values, left=start,  height=1, width=df_life_events['num_of_days'].values, color=df_life_events['colour'], ec='black')

# custom_lines = [Line2D([0], [0], color='indianred', lw=4),
#                Line2D([0], [0], color='orange', lw=4),
#                Line2D([0], [0], color='lightgreen', lw=4),
#                Line2D([0], [0], color='teal', lw=4),
#                Line2D([0], [0], color='blueviolet', lw=4),
#                Line2D([0], [0], color='navy', lw=4),
#                Line2D([0], [0], color='skyblue', lw=4),
#                Line2D([0], [0], color= 'black', lw=4, marker='*', linestyle='')]

# ax.legend(custom_lines, ['Personal', 'Health', 'Work', 'Financial','Weather','Societal','Other', 'FB Events'], loc='upper left')

# ax.spines['bottom'].set_position('zero')
# ax.spines['top'].set_color('none')
# ax.spines['right'].set_color('none')
# ax.spines['left'].set_color('none')
# ax.tick_params(axis='x', length=5)
# # ax.set_yticks([])  # turn off the yticks

# # ymin, ymax = ax.get_ylim()
# ax.set_xlim(xmin, xmax)
# ax.set_ylim(ymin-10, ymax+30) # make room for the legend

# # plt.tight_layout()
# plt.show()

In [633]:
# def plot_life_events_with_stress(type_of_event):
#     df_life_events_single_event = df_life_events[df_life_events['life_event_type'] == type_of_event]
#     fig, ax = plt.subplots(figsize=(40,40))

#     ymin = min(min(df_stress['y_axis'].values), min(df_life_events_single_event['y_axis'].values))
#     ymax = min(max(df_stress['y_axis'].values), max(df_life_events_single_event['y_axis'].values))

#     start = [mdates.date2num(v) for v in df_life_events_single_event['UpdatedBeginDate_time'].values]

#     stress_y = []
#     stress_x = []
#     for index, row in df_stress.iterrows():
#         if(not(pd.isna(row['stress.d'])) and int(row['stress.d']) != 0):
#             stress_x.append(row['day_time'])
#             stress_y.append(row['y_axis'])

#     xmin = max(min(stress_x), min(df_life_events_single_event['UpdatedBeginDate_time'].values))
#     xmax = min(max(stress_x), max(df_life_events_single_event['UpdatedBeginDate_time'].values))


#     # plt.plot(df_facebook['created_date'].values, df_facebook['y_axis'].values, linestyle = 'None', marker='*', color='black', label='line with marker')
#     plt.plot(stress_x, stress_y,  linestyle = 'None', marker='^', color='grey', label='line with marker')
#     plt.barh(y=df_life_events_single_event['y_axis'].values, left=start,  height=1, width=df_life_events_single_event['num_of_days'].values, color=df_life_events_single_event['colour'], ec='black')

#     custom_lines = [Line2D([0], [0], color='indianred', lw=4),
#                    Line2D([0], [0], color='orange', lw=4),
#                    Line2D([0], [0], color='lightgreen', lw=4),
#                    Line2D([0], [0], color='teal', lw=4),
#                    Line2D([0], [0], color='blueviolet', lw=4),
#                    Line2D([0], [0], color='navy', lw=4),
#                    Line2D([0], [0], color='skyblue', lw=4),
#                    Line2D([0], [0], color= 'black', lw=4, marker='^', linestyle='')]

#     ax.legend(custom_lines, ['Personal', 'Health', 'Work', 'Financial','Weather','Societal','Other', 'Reported Stress'], loc='upper left')

#     ax.spines['bottom'].set_position('zero')
#     ax.spines['top'].set_color('none')
#     ax.spines['right'].set_color('none')
#     ax.spines['left'].set_color('none')
#     ax.tick_params(axis='x', length=5)
#     # ax.set_yticks([])  # turn off the yticks

#     # ymin, ymax = ax.get_ylim()
#     ax.set_xlim(xmin, xmax)
#     ax.set_ylim(ymin-10, ymax+30) # make room for the legend

#     # plt.tight_layout()
#     plt.show()

In [634]:
# plot_life_events_with_stress('work')

In [635]:
# plot_life_events_with_stress('personal')

In [636]:
# plot_life_events_with_stress('health')

In [637]:
# plot_life_events_with_stress('financial')

In [638]:
# plot_life_events_with_stress('weather')

In [639]:
# plot_life_events_with_stress('societal')

In [640]:
# plot_life_events_with_stress('other')

In [641]:
# def plot_individual(y_axis_id):
#     data = df_stress[df_stress['y_axis'] == y_axis_id]
#     data = data.sort_values('day_time')
#     selected_life_events = df_life_events[df_life_events['y_axis'] == y_axis_id]
#     fig, ax = plt.subplots(figsize=(20,20))

#     start = [mdates.date2num(v) for v in selected_life_events['UpdatedBeginDate_time'].values]

#     plt.plot(data['day_time'].values, data['stress.d'], marker='^', color='grey', label='line with marker')
#     plt.barh(y=selected_life_events['y_axis'].values, left=start,  height=1, width=selected_life_events['num_of_days'].values, color=selected_life_events['colour'], ec='black')

#     custom_lines = [Line2D([0], [0], color='indianred', lw=4),
#                    Line2D([0], [0], color='orange', lw=4),
#                    Line2D([0], [0], color='lightgreen', lw=4),
#                    Line2D([0], [0], color='teal', lw=4),
#                    Line2D([0], [0], color='blueviolet', lw=4),
#                    Line2D([0], [0], color='navy', lw=4),
#                    Line2D([0], [0], color='skyblue', lw=4),
#                    Line2D([0], [0], color= 'black', lw=4, marker='^', linestyle='')]

#     ax.legend(custom_lines, ['Personal', 'Health', 'Work', 'Financial','Weather','Societal','Other', 'Reported Stress'], loc='upper left')

#     ax.spines['bottom'].set_position('zero')
#     ax.spines['top'].set_color('none')
#     ax.spines['right'].set_color('none')
#     ax.spines['left'].set_color('none')
#     ax.tick_params(axis='x', length=5)
#     # ax.set_yticks([])  # turn off the yticks

#     # ymin, ymax = ax.get_ylim()
# #     ax.set_xlim(xmin, xmax)
# #     ax.set_ylim(ymin-10, ymax+30) # make room for the legend

#     # plt.tight_layout()
#     plt.show()

In [642]:
# plot_individual(4)

In [644]:
input_life_events = df_life_events[['snapshot_id','y_axis', 'UpdatedBeginDate', 'UpdatedEndDate', 'significance', 'valence', 'life_event_type']]
merged_data = pd.merge(input_life_events, demographics_data, how="inner", on=["snapshot_id"])

le_valence = LabelEncoder()
le_valence.fit(merged_data['valence'].values)

le_confidence = LabelEncoder()
le_confidence.fit(merged_data['significance'].values)

le_event_type = LabelEncoder()
le_event_type.fit(merged_data['life_event_type'].values)

gender_encoder = LabelEncoder()
gender_encoder.fit(merged_data['gender'].values)

merged_data['valence_label'] = merged_data.apply(lambda x: le_valence.transform([x['valence']])[0], axis=1)
merged_data['significance_label'] = merged_data.apply(lambda x: le_confidence.transform([x['significance']])[0], axis=1)
merged_data['event_type_label'] = merged_data.apply(lambda x: le_event_type.transform([x['life_event_type']])[0], axis=1)
merged_data['gender_label'] = merged_data.apply(lambda x: gender_encoder.transform([x['gender']])[0], axis=1)

merged_data = merged_data.drop(columns=['y_axis_x', 'y_axis_y', 'valence', 'significance', 'life_event_type', 'gender'])
selected_df_stress = df_stress[['snapshot_id', 'day_time', 'stress.d']]
final_input_dataset = []

for i in merged_data.values:
    end_date = datetime.datetime.strptime(i[2], "%Y-%m-%d").date()
    start_date = datetime.datetime.strptime(i[1], "%Y-%m-%d").date()
    days = (end_date - start_date).days
    for j in range(days):
        day = start_date+timedelta(days=j)
        x = list(selected_df_stress[((selected_df_stress['snapshot_id'] == i[0]) & (selected_df_stress['day_time'] == day))].values)
        stres = 0
        if(len(x) != 0):
            stres = x[0][-1]
        s_d = []
        s_d.extend(i)
        s_d.append(stres)
        s_d.append(day)
        final_input_dataset.append(s_d)
    
final_input_dataset = pd.DataFrame(final_input_dataset, columns = [
    'snapshot_id', 'UpdatedBeginDate', 'UpdatedEndDate', 'age',
       'shipley.vocab', 'shipley.abs', 'openness', 'conscientiousness',
       'extraversion', 'agreeableness', 'neuroticism', 'pos.affect',
       'neg.affect', 'stai.trait', 'valence_label', 
       'significance_label', 'event_type_label', 'gender_label',
       'stress.d','exact_day'])                                   

In [654]:
X_input = final_input_dataset.drop(columns=['snapshot_id', 'UpdatedBeginDate', 'UpdatedEndDate', 'exact_day'])

X_input = X_input.dropna()
X_input['gender_label'] = X_input['gender_label'].astype("category")
X_input['event_type_label'] = X_input['event_type_label'].astype("category")
print(X_input.dtypes)

age                    float64
shipley.vocab          float64
shipley.abs            float64
openness               float64
conscientiousness      float64
extraversion           float64
agreeableness          float64
neuroticism            float64
pos.affect             float64
neg.affect             float64
stai.trait             float64
valence_label            int64
significance_label       int64
event_type_label      category
gender_label          category
stress.d               float64
dtype: object


In [648]:
X_input['stress.d'].value_counts()

2.0    658
3.0    589
1.0    494
4.0    133
5.0     23
Name: stress.d, dtype: int64

In [652]:
# spector_data.exog = sm.add_constant(X_input['stress.d'], prepend=False)
try_x = X_input.drop(columns=['stress.d'])
X_cons = sm.add_constant(try_x)
mod = sm.OLS(X_input['stress.d'], try_x)
res = mod.fit()
print(res.summary())

                                 OLS Regression Results                                
Dep. Variable:               stress.d   R-squared (uncentered):                   0.864
Model:                            OLS   Adj. R-squared (uncentered):              0.863
Method:                 Least Squares   F-statistic:                              795.5
Date:                Thu, 14 Apr 2022   Prob (F-statistic):                        0.00
Time:                        17:27:28   Log-Likelihood:                         -2479.3
No. Observations:                1897   AIC:                                      4989.
Df Residuals:                    1882   BIC:                                      5072.
Df Model:                          15                                                  
Covariance Type:            nonrobust                                                  
                         coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------